In [ ]:
# --- Imports ---
import pandas as pd  # For working with structured data (tables)
import numpy as np  # For numerical operations
import yfinance as yf  # To get financial data from Yahoo Finance
import requests  # To send HTTP requests to APIs
import time  # To pause the program if needed or add delays
from datetime import datetime, timedelta  # To handle and format dates
from scipy.stats import gmean  # To calculate geometric mean for performance factor calculation
from IPython.display import display  # To neatly display tables in Jupyter notebook

# --- Configuration ---
FMP_API_KEY = "aeKO0VHKnbpwpeAogoL99dNsaBepolWS"  # Use API key, can make another one

# List of investment funds or securities to analyze
TICKERS = ["BND", "VFIFX", "VBTLX", "VBIAX"]  # Example: bonds and target date funds
# BND: Vanguard Total Bond Market ETF
# VFIFX: Vanguard Target Retirement 2050 Fund
# VBTLX: Vanguard Total Bond Market Index Fund Admiral Shares
# VBIAX: Vanguard Balanced Index Fund Admiral Shares


# The date the portfolio analysis is based on (i.e., "how did we do up until this date?")
AS_OF_DATE = "2025-03-31"  # Can change as needed

# Hard-coded portfolio code, can fix later but copied what was already in database
DEFAULT_PORTFOLIO_CODE = "VQO59WR8"

# --- Function: Get Asset Type and Historical Price Data ---
def get_security_type_and_data_yf(ticker, start_date, end_date):
    """
    Pulls price data for a given ticker from Yahoo Finance. Tries to figure out whether it's a stock or bond.
    Returns data, inception date, and type of security.
    """

    # Initializes dictionary to store results from function
    result = {
        'price_data': None, # This will hold the price data (if found)
        'inception_date': None, # Date when the fund/stock started trading
        'security_type': 'unknown' # Will be updated to 'bond' or 'stock' if we can figure it out
    }

    try:
        # Get ticker info
        ticker_obj = yf.Ticker(ticker)  # Creates a Yahoo Finance object for the given ticker
        info = ticker_obj.info  # Pulls general info tied to ticker

        # Try to identify whether it's a bond or stock based on Yahoo's category info
        if 'quoteType' in info:
            if info['quoteType'] in ('ETF', 'MUTUALFUND'):
                # For ETFs and Mutual Funds, check the category/asset class
                category = info.get('category', '').lower() # Get the type of investment (e.g., fixed income)
                if any(bond_term in category for bond_term in ('bond', 'fixed income', 'debt')):
                    result['security_type'] = 'bond'
                else:
                    result['security_type'] = 'stock'
            else:
                result['security_type'] = 'stock'

        # Get historical price data from Yahoo to determine its inception date
        earliest_df = ticker_obj.history(period="max")

        # Validation check for if no data is found
        if earliest_df.empty:
            print(f"No data found for {ticker} from Yahoo Finance")
            return result

        # First available date is used as the inception date
        inception_date = earliest_df.index.min().strftime('%Y-%m-%d')
        result['inception_date'] = inception_date

        # Get the data within the desired start and end dates
        df = ticker_obj.history(start=start_date, end=end_date)

        # Extracts closing prices from the data
        series = df["Close"]

        # Convert to DataFrame with date as column
        df_prices = series.to_frame(name="price")  # Rename to just "price"

        # Reset index to get Date as column, not an index
        df_prices.index.name = "Date"
        df_prices = df_prices.reset_index()

        result['price_data'] = df_prices

        # If still can't figure out if it's a bond, guess based on the ticker name
        if result['security_type'] == 'unknown':
            # Check ticker name for bond indicators
            if any(bond_term in ticker.upper() for bond_term in ('BOND', 'BND', 'FIXED', 'TREASURY', 'CORP')):
                result['security_type'] = 'bond'
            else:
                result['security_type'] = 'stock'

        return result

    # Error handling
    except Exception as e:
        print(f"Error retrieving Yahoo Finance data for {ticker}: {e}")
        return result

# --- Function: get_security_type_and_data_fmp ---
def get_security_type_and_data_fmp(ticker, api_key):
    """
    Similar to the previous function, but uses FinancialModelingPrep (FMP) instead of Yahoo.
    Tries to find what type of investment it is and gather its price history.
    """

    # Same dictionary structure to store data
    result = {
        'price_data': None,
        'inception_date': None,
        'security_type': 'unknown'
    }

    # Set up API
    base_url = "https://financialmodelingprep.com/api/v3"
    hist_url = f"{base_url}/historical-price-full/{ticker}?apikey={api_key}"
    profile_url = f"{base_url}/profile/{ticker}?apikey={api_key}"

    try:
        # Try to get the profile data first
        try:
            profile_response = requests.get(profile_url)  # Makes an HTTP request
            profile_response.raise_for_status()  # Validates response
            profile_data = profile_response.json()  # Parses JSON

            if profile_data and len(profile_data) > 0:  # Checks if profile data was returned
                # Determine security type from profile
                sector = profile_data[0].get('sector', '').lower()
                industry = profile_data[0].get('industry', '').lower()

                # Combines sector & industry strings, checks for bond-related terms and classifies it
                if any(bond_term in sector + industry for bond_term in ('bond', 'fixed income', 'debt', 'treasury')):
                    result['security_type'] = 'bond'
                else:
                    result['security_type'] = 'stock'

                # Get IPO date, stored as inception date
                ipo_date = profile_data[0].get('ipoDate')
                if ipo_date:
                    result['inception_date'] = ipo_date
                    print(f"Found IPO date for {ticker}: {ipo_date}")
        except Exception as e:
            print(f"Error retrieving profile data for {ticker}: {e}")

        # Get historical price data
        response = requests.get(hist_url)
        response.raise_for_status()
        hist_data = response.json()

        if 'historical' in hist_data and hist_data['historical']:
            # Convert to DataFrame and format
            hist_df = pd.DataFrame(hist_data['historical'])
            hist_df = hist_df.rename(columns={'date': 'Date', 'close': 'price'})
            result['price_data'] = hist_df

            # If no IPO date found in profile, use earliest historical date as a substitute
            if not result['inception_date']:
                earliest_date = min(hist_df['Date'])
                result['inception_date'] = earliest_date
                print(f"Using earliest historical date for {ticker}: {earliest_date}")

            # If security type is still unknown, check ticker name
            if result['security_type'] == 'unknown':
                if any(bond_term in ticker.upper() for bond_term in ('BOND', 'BND', 'FIXED', 'TREASURY', 'CORP')):
                    result['security_type'] = 'bond'
                else:
                    result['security_type'] = 'stock'

        return result

    except Exception as e:
        print(f"Error retrieving FMP data for {ticker}: {e}")
        return result

# --- Function: get_security_data ---
def get_security_data(ticker, start_date, end_date, api_key):
    """
    Tries to get security data using both FMP and Yahoo.
    Starts with FMP, and if that fails, uses Yahoo as a backup.
    FMP has bond data, which is why it is used first. Yahoo does not.
    """

    print(f"Getting data for {ticker}...")

    # First try FMP API
    print(f"Trying Financial Model Prep API for {ticker}...")
    fmp_data = get_security_type_and_data_fmp(ticker, api_key)

    # If we got good data from FMP, use it
    if fmp_data['price_data'] is not None and not fmp_data['price_data'].empty:
        print(f"Successfully got data for {ticker} from FMP. Security type: {fmp_data['security_type']}")
        return fmp_data

    # Otherwise, try Yahoo Finance
    print(f"Trying Yahoo Finance for {ticker}...")
    yf_data = get_security_type_and_data_yf(ticker, start_date, end_date)

    # If no data is found from Yahoo, prints error message
    if yf_data['price_data'] is not None and not yf_data['price_data'].empty:
        print(f"Successfully got data for {ticker} from Yahoo Finance. Security type: {yf_data['security_type']}")
        return yf_data

    print(f"Could not get data for {ticker} from either source")
    return {'price_data': None, 'inception_date': None, 'security_type': 'unknown'}

# --- Data Retrieval Functions ---
def calculate_geometric_mean_returns(prices):
    """
    Calculate returns using geometric mean.

    Args:
        prices: List or array of price values

    Returns:
        float: Geometric mean return
    """

    # Validation check: need at least 2 prices to calculate returns
    if len(prices) < 2:
        return 0  # Returns 0 if not enough prices

    # Calculate returns between each consecutive pair of prices
    returns = []  # List to store return ratios
    for i in range(1, len(prices)):
        prev_price = prices[i-1]
        curr_price = prices[i]
        if prev_price > 0:
            returns.append(curr_price / prev_price)  # Formula: return = current price / previous price

    if not returns:
        return 0

    try:
        # Uses gmean to calculate geometric mean of return ratios
        # Formula: (r₁ × r₂ × ... × rₙ)^(1/n)
        geo_mean = gmean(returns) - 1  # Subtract 1 to convert ratio to a percentage
        # Cap between -100% and +100%
        return max(min(geo_mean, 1.0), -1.0)  # Applies bounds to GMEAN so between -1 and 1, has to be within that range
    except Exception as e:  # Error handling
        print(f"Error calculating geometric mean: {e}")
        return 0

# --- Data Retrieval Functions ---
def create_portfolio_performance_table(tickers, as_of=None, portfolio_code=None, api_key=None):
    """
    Create a portfolio performance table for a list of tickers.
    Each security's inception date is respected in performance calculations.

    Args:
        tickers: List of tickers to include in the portfolio
        as_of: Date to calculate performance up to (default: today)
        portfolio_code: Code to use for the portfolio (default: DEFAULT_PORTFOLIO_CODE)
        api_key: Financial Model Prep API key

    Returns:
        DataFrame with portfolio performance data matching Snowflake schema
    """

    # Validation check to make sure tickers list isn't empty, especially since using HTML to populate list later
    if not tickers:
        print("No tickers provided")
        return pd.DataFrame()

    # Sets default API key if none is provided: might be unecessary, can delete later
    if api_key is None:
        api_key = FMP_API_KEY

    # Default portfolio code is what is currently in DB, can change it so not all the same
    # if portfolio_code is None:
    # portfolio_code = DEFAULT_PORTFOLIO_CODE
    portfolio_code = DEFAULT_PORTFOLIO_CODE

    # Handle as_of date: if none provided, then use today's date
    if as_of is None:
        as_of = datetime.now().strftime("%Y-%m-%d")
    else:
        as_of = pd.to_datetime(as_of).strftime("%Y-%m-%d")

    print(f"Using as-of date: {as_of}")

    # Get price data and inception dates for all tickers, store in dictionaries
    all_ticker_data = {}
    ticker_inception_dates = {}
    security_types = {}

    # Loops through each ticker in list, pull data from Yahoo Finance or Financial Modeling Prep
    for ticker in tickers:
        security_data = get_security_data(ticker, "1900-01-01", as_of, api_key)  # Hard code a start date to get all possible historical data

        # Checks if valid price data retrieved
        if security_data['price_data'] is not None and not security_data['price_data'].empty:
            price_data = security_data['price_data']

            # Ensure date is datetime format
            if isinstance(price_data['Date'].iloc[0], str):
                price_data['Date'] = pd.to_datetime(price_data['Date'])

            # Store data in respective dictionaries
            all_ticker_data[ticker] = price_data
            security_types[ticker] = security_data['security_type']

            # Store inception date for each ticker
            if security_data['inception_date']:
                inception_date_dt = pd.to_datetime(security_data['inception_date'])
                ticker_inception_dates[ticker] = inception_date_dt
                print(f"Inception date for {ticker}: {inception_date_dt.strftime('%Y-%m-%d')}")

    # Validation Check: ensures at least one ticker has valid data
    if not all_ticker_data:
        print("No valid data found for any ticker")
        return pd.DataFrame()

    # Determine portfolio inception date (latest of all security inception dates)
    if ticker_inception_dates:
        # Sort by date and print all inception dates for clarity
        sorted_inception_dates = sorted([(t, d) for t, d in ticker_inception_dates.items()], key=lambda x: x[1]) # Create list of ticker, inception date tuples to prevent values from being altered
        print("\nAll inception dates:")
        for ticker, date in sorted_inception_dates:
            print(f"  {ticker}: {date.strftime('%Y-%m-%d')}")

        # Portfolio inception is the latest date when all securities existed
        # Finds latest inception date among all securities and sets it as portfolio inception date
        # Portfolio inception date can't be before the securities were created

        latest_ticker, latest_date = max(sorted_inception_dates, key=lambda x: x[1])
        portfolio_inception = latest_date
        print(f"Portfolio inception date (latest security): {latest_ticker} on {portfolio_inception.strftime('%Y-%m-%d')}")
    else:
        # Fallback if no inception dats were found: use 10 years before as_of
        portfolio_inception = pd.to_datetime(as_of) - pd.DateOffset(years=10)
        print(f"No inception dates found. Using fallback date: {portfolio_inception.strftime('%Y-%m-%d')}")

    # Create combined monthly price table for portfolio calculations
    all_dates = []
    for ticker, data in all_ticker_data.items():
        # Only use dates after portfolio inception
        valid_dates = data[data['Date'] >= portfolio_inception]['Date'].tolist()
        all_dates.extend(valid_dates)

    # Get unique month-end dates
    all_dates = pd.to_datetime(sorted(set(all_dates)))
    all_dates = pd.Series(all_dates).dt.to_period('M').unique()
    monthly_dates = pd.Series([pd.Timestamp(dt.to_timestamp()) for dt in all_dates]).sort_values()

    # Create monthly price table for portfolio using the month-end dates, might need to change if this isn't correct interpretation of HISTORYDATE
    monthly_table = pd.DataFrame({"HISTORYDATE": monthly_dates})

    # Add columns for each ticker's price
    for ticker, data in all_ticker_data.items():
        # Filter data to be after portfolio inception
        ticker_data = data[data['Date'] >= portfolio_inception]

        # Group by month and get last price of each month: takes month-end price
        ticker_data['month'] = ticker_data['Date'].dt.to_period('M') # Assigns each price data point to its month period
        monthly_prices = ticker_data.groupby('month')['price'].last().reset_index() # Groups by month and takes last price of each month
        monthly_prices['month'] = monthly_prices['month'].apply(lambda x: pd.Timestamp(x.to_timestamp())) # Converts period objects back to timestamps

        # Join to the monthly table
        # Series with prices indexed by their dates, reindexes to match the dates in monthly_table
        monthly_lookup = pd.Series(monthly_prices['price'].values, index=monthly_prices['month']).reindex(monthly_table['HISTORYDATE'])
        monthly_table[ticker] = monthly_lookup.values  # Add Series to a new column in monthly_table named after the ticker

    # Calculate portfolio price (average of all securities)
    price_columns = [col for col in monthly_table.columns if col != 'HISTORYDATE']
    if price_columns:
        # Fill missing values with forward/backward fill to ensure complete price series
        monthly_table[price_columns] = monthly_table[price_columns].fillna(method='ffill').fillna(method='bfill')

        # Calculate portfolio's average price (equally weighted): average price across all tickers for each month
        monthly_table["price"] = monthly_table[price_columns].mean(axis=1)

    # Initialize lists for performance records
    security_performance_records = []  # Individual security records
    portfolio_performance_records = []  # Portfolio-level records

    # --- Part 1: Create individual security performance records ---
    for ticker in tickers:
        if ticker not in all_ticker_data or ticker not in ticker_inception_dates:
            continue

        ticker_data = all_ticker_data[ticker] # Gets price data for ticker
        ticker_inception = ticker_inception_dates[ticker] # Gets inception date for ticker

        # Filter data to be after this ticker's inception date
        # It might be earlier than the portfolio's inception date
        ticker_monthly_data = ticker_data[ticker_data['Date'] >= ticker_inception]

        # Group by month and get last price of each month
        ticker_monthly_data['month'] = ticker_monthly_data['Date'].dt.to_period('M')
        monthly_prices = ticker_monthly_data.groupby('month')['price'].last().reset_index()
        monthly_prices['month'] = monthly_prices['month'].apply(lambda x: pd.Timestamp(x.to_timestamp()))
        monthly_prices = monthly_prices.rename(columns={'month': 'HISTORYDATE'})

        # Add monthly performance records for this ticker (need at least 2 months of data)
        for i in range(2, len(monthly_prices) + 1):  # Calculate for each month using data up to that month
            # Get prices from inception up to current month
            prices_up_to_month = monthly_prices['price'].iloc[:i].values

            # Calculate geometric mean return using prices from inception to current month
            performance_factor = calculate_geometric_mean_returns(prices_up_to_month)

            # Current month date
            current_date = monthly_prices['HISTORYDATE'].iloc[i-1]

            # Set performance inception date to first date in price series, typically the ticker's inception date
            perf_start_date = monthly_prices['HISTORYDATE'].iloc[0]

            security_performance_records.append({
                'PORTFOLIOCODE': DEFAULT_PORTFOLIO_CODE,  # Portfolio code corresponding to ticker, can change from default to later
                'HISTORYDATE': current_date,
                'CURRENCYCODE': 'USD',
                'CURRENCY': 'US Dollar',
                'PERFORMANCECATEGORY': 'Asset Class',
                'PERFORMANCECATEGORYNAME': 'Total Portfolio',
                'PERFORMANCETYPE': 'Portfolio Gross',
                'PERFORMANCEINCEPTIONDATE': perf_start_date,
                'PORTFOLIOINCEPTIONDATE': ticker_inception,
                'PERFORMANCEFREQUENCY': 'M',
                'PERFORMANCEFACTOR': float(performance_factor)
            })

        # Calculate multi-period performance for this ticker
        latest_date = monthly_prices['HISTORYDATE'].max() if not monthly_prices.empty else None

        # Check if there is a valid latest date
        if latest_date is not None:
            # Loops through standard performance periods
            for period_name, years in [('1Y', 1), ('3Y', 3), ('5Y', 5), ('10Y', 10)]:
                # Calculate start date for this period by subtracting number of years from the latest date
                perf_start_date = latest_date - pd.DateOffset(years=years)

                # Validation check: ensures don't calculate performance for periods before the ticker existed
                if perf_start_date < ticker_inception:
                    print(f"Skipping {period_name} calculation for {ticker} as it would start before security inception")
                    continue

                # Get prices within this period
                period_subset = monthly_prices[monthly_prices['HISTORYDATE'] >= perf_start_date]

                # Check if at least 2 price points in period
                if len(period_subset) >= 2:
                    # Calculate geometric mean return for the period
                    period_prices = period_subset['price'].values
                    annualized_return = calculate_geometric_mean_returns(period_prices)

                    # Creates a performance record for multi-period calculation at security level
                    security_performance_records.append({
                        'PORTFOLIOCODE': ticker,
                        'HISTORYDATE': latest_date,  # Latest date
                        'CURRENCYCODE': 'USD',
                        'CURRENCY': 'US Dollar',
                        'PERFORMANCECATEGORY': 'Asset Class',      # From database
                        'PERFORMANCECATEGORYNAME': 'Total Portfolio', # From database
                        'PERFORMANCETYPE': 'Portfolio Gross',         # From database
                        'PERFORMANCEINCEPTIONDATE': perf_start_date,  # Start date for this period
                        'PORTFOLIOINCEPTIONDATE': ticker_inception,
                        'PERFORMANCEFREQUENCY': f"{period_name} Annualized",  # Annualized period
                        'PERFORMANCEFACTOR': float(annualized_return)  # Annualized return
                    })

    # --- Part 2: Create portfolio-level performance records ---

    # Add monthly portfolio performance records (need at least 2 months of data)
    if len(monthly_table) >= 2:
        for i in range(2, len(monthly_table) + 1):  # Calculate for each month using data up to that month
            # Get prices up to this month
            prices_up_to_month = monthly_table['price'].iloc[:i].values

            # Calculate geometric mean return
            performance_factor = calculate_geometric_mean_returns(prices_up_to_month)

            # Current month date
            current_date = monthly_table['HISTORYDATE'].iloc[i-1]

            # Performance inception is the first date in the portfolio's price series
            perf_start_date = monthly_table['HISTORYDATE'].iloc[0]

            # Creates a performance record for this month at the portfolio level
            portfolio_performance_records.append({
                'PORTFOLIOCODE': portfolio_code,
                'HISTORYDATE': current_date,
                'CURRENCYCODE': 'USD',
                'CURRENCY': 'US Dollar',
                'PERFORMANCECATEGORY': 'Asset Class',  # From database
                'PERFORMANCECATEGORYNAME': 'Total Portfolio',    # From database
                'PERFORMANCETYPE': 'Portfolio Gross',  # From database
                'PERFORMANCEINCEPTIONDATE': perf_start_date,
                'PORTFOLIOINCEPTIONDATE': portfolio_inception,
                'PERFORMANCEFREQUENCY': 'M',
                'PERFORMANCEFACTOR': float(performance_factor)
            })

    # Calculate multi-period performance for the portfolio
    latest_date = monthly_table['HISTORYDATE'].max() if not monthly_table.empty else None

    # Check if latest date is valid
    if latest_date is not None:
        for period_name, years in [('1Y', 1), ('3Y', 3), ('5Y', 5), ('10Y', 10)]:
            # Calculate start date for this period's measurement
            perf_start_date = latest_date - pd.DateOffset(years=years)

            # Validation check: ensures don't calculate performance for periods before the portfolio existed
            if perf_start_date < portfolio_inception:
                print(f"Skipping {period_name} calculation for portfolio as it would start before portfolio inception date")
                continue

            # Get prices within this period
            period_subset = monthly_table[monthly_table['HISTORYDATE'] >= perf_start_date]

            # Check if at least 2 price points in period
            if len(period_subset) >= 2:
                # Calculate geometric mean return for the period
                period_prices = period_subset['price'].values
                annualized_return = calculate_geometric_mean_returns(period_prices)

                # Creates a performance record for multi-period calculation at the portfolio level
                portfolio_performance_records.append({
                    'PORTFOLIOCODE': portfolio_code,
                    'HISTORYDATE': latest_date,
                    'CURRENCYCODE': 'USD',
                    'CURRENCY': 'US Dollar',
                    'PERFORMANCECATEGORY': 'Asset Class',            # From database
                    'PERFORMANCECATEGORYNAME': 'Total Portfolio',   # From database
                    'PERFORMANCETYPE': 'Portfolio Gross',           # From database
                    'PERFORMANCEINCEPTIONDATE': perf_start_date,
                    'PORTFOLIOINCEPTIONDATE': portfolio_inception,
                    'PERFORMANCEFREQUENCY': f"{period_name} Annualized",
                    'PERFORMANCEFACTOR': float(annualized_return)
                })

    # --- Part 3: Combine all records and create DataFrame ---

    # Combines the individual security records and portfolio records into a single list
    all_records = security_performance_records + portfolio_performance_records

    if all_records:
        result_df = pd.DataFrame(all_records)

        # Ensure all columns are present and in the correct order
        columns = [
            'PORTFOLIOCODE', 'HISTORYDATE', 'CURRENCYCODE', 'CURRENCY',
            'PERFORMANCECATEGORY', 'PERFORMANCECATEGORYNAME', 'PERFORMANCETYPE',
            'PERFORMANCEINCEPTIONDATE', 'PORTFOLIOINCEPTIONDATE',
            'PERFORMANCEFREQUENCY', 'PERFORMANCEFACTOR'
        ]

        # Checks for any missing columns and adds them with default values:
        for col in columns:
            if col not in result_df.columns:
                if col in ['HISTORYDATE', 'PERFORMANCEINCEPTIONDATE', 'PORTFOLIOINCEPTIONDATE']:
                    result_df[col] = portfolio_inception
                elif col == 'PERFORMANCEFACTOR':
                    result_df[col] = 0.0
                else:
                    result_df[col] = ''

        # Return DataFrame with columns in the correct order
        return result_df[columns]
    else:
        # Return empty DataFrame with correct columns
        return pd.DataFrame(columns=[
            'PORTFOLIOCODE', 'HISTORYDATE', 'CURRENCYCODE', 'CURRENCY',
            'PERFORMANCECATEGORY', 'PERFORMANCECATEGORYNAME', 'PERFORMANCETYPE',
            'PERFORMANCEINCEPTIONDATE', 'PORTFOLIOINCEPTIONDATE',
            'PERFORMANCEFREQUENCY', 'PERFORMANCEFACTOR'
        ])

def main():
    try:
        print("\n----- Portfolio Performance Table Generator -----\n")

        # Use predefined tickers from the TICKERS list at the top of the file
        tickers = TICKERS

        # Use predefined as-of date from the AS_OF_DATE variable
        as_of_date = AS_OF_DATE

        # Uses portfolio code from databse, can change later if needed
        portfolio_code = DEFAULT_PORTFOLIO_CODE

        #print(f"Creating portfolio performance table for tickers: {tickers}")
        #print(f"Using as-of date: {as_of_date}")
        #print(f"Using portfolio code: {portfolio_code}")

        # Create portfolio performance table
        try:
            performance_table = create_portfolio_performance_table(
                tickers=tickers,
                as_of=as_of_date,
                portfolio_code=portfolio_code,
                api_key=FMP_API_KEY
            )
            print(f"Successfully created performance table with shape: {performance_table.shape}")
        except Exception as e:
            print(f"Error creating portfolio performance table: {str(e)}")
            performance_table = pd.DataFrame()

        # Display the table
        print("\nPortfolio Performance Table:")
        display(performance_table.head())

        return performance_table

    except Exception as e:
        print(f"Error in main function: {str(e)}")
        return pd.DataFrame()

if __name__ == "__main__":
    result = main()
    print("Program completed")


----- Portfolio Performance Table Generator -----

Using as-of date: 2025-03-31
Getting data for BND...
Trying Financial Model Prep API for BND...
Found IPO date for BND: 2007-04-10
Successfully got data for BND from FMP. Security type: bond
Inception date for BND: 2007-04-10
Getting data for VFIFX...
Trying Financial Model Prep API for VFIFX...
Found IPO date for VFIFX: 2006-06-07
Successfully got data for VFIFX from FMP. Security type: stock
Inception date for VFIFX: 2006-06-07
Getting data for VBTLX...
Trying Financial Model Prep API for VBTLX...
Found IPO date for VBTLX: 2001-11-12
Successfully got data for VBTLX from FMP. Security type: bond
Inception date for VBTLX: 2001-11-12
Getting data for VBIAX...
Trying Financial Model Prep API for VBIAX...
Found IPO date for VBIAX: 2000-11-13
Successfully got data for VBIAX from FMP. Security type: stock
Inception date for VBIAX: 2000-11-13

All inception dates:
  VBIAX: 2000-11-13
  VBTLX: 2001-11-12
  VFIFX: 2006-06-07
  BND: 2007-04-10

/tmp/ipython-input-25-3129477147.py:368: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  monthly_table[price_columns] = monthly_table[price_columns].fillna(method='ffill').fillna(method='bfill')


,PORTFOLIOCODE,HISTORYDATE,CURRENCYCODE,CURRENCY,PERFORMANCECATEGORY,PERFORMANCECATEGORYNAME,PERFORMANCETYPE,PERFORMANCEINCEPTIONDATE,PORTFOLIOINCEPTIONDATE,PERFORMANCEFREQUENCY,PERFORMANCEFACTOR
0,VQO59WR8,2020-08-01,USD,US Dollar,Asset Class,Total Portfolio,Portfolio Gross,2020-07-01,2007-04-10,M,0.003710
1,VQO59WR8,2020-09-01,USD,US Dollar,Asset Class,Total Portfolio,Portfolio Gross,2020-07-01,2007-04-10,M,-0.002195
2,VQO59WR8,2020-10-01,USD,US Dollar,Asset Class,Total Portfolio,Portfolio Gross,2020-07-01,2007-04-10,M,-0.002932
3,VQO59WR8,2020-11-01,USD,US Dollar,Asset Class,Total Portfolio,Portfolio Gross,2020-07-01,2007-04-10,M,-0.003930
4,VQO59WR8,2020-12-01,USD,US Dollar,Asset Class,Total Portfolio,Portfolio Gross,2020-07-01,2007-04-10,M,-0.001941


Program completed
